In [1]:
from os import listdir
from os.path import isfile, join
import os, sys
sys.path.append('pytorch_sol2/')
sys.path

['/Users/mohamedzayyanvariawa/Documents/machinelearningprojects/Computer Vision/Covid pneumonia',
 '/opt/anaconda3/envs/pytorchenv/lib/python39.zip',
 '/opt/anaconda3/envs/pytorchenv/lib/python3.9',
 '/opt/anaconda3/envs/pytorchenv/lib/python3.9/lib-dynload',
 '',
 '/opt/anaconda3/envs/pytorchenv/lib/python3.9/site-packages',
 '/opt/anaconda3/envs/pytorchenv/lib/python3.9/site-packages/aeosa',
 '/opt/anaconda3/envs/pytorchenv/lib/python3.9/site-packages/locket-0.2.1-py3.9.egg',
 '/opt/anaconda3/envs/pytorchenv/lib/python3.9/site-packages/IPython/extensions',
 '/Users/mohamedzayyanvariawa/.ipython',
 'pytorch_sol2/']

In [2]:
import pandas as pd
import numpy as np
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from skimage import io, transform
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import torch.nn.functional as F

In [3]:
from pneumoniaDataset import *

In [4]:
device = torch.device('cpu')

In [5]:
transformations = transforms.Compose([transforms.ToTensor(),
    transforms.Resize((1000, 1000)),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [6]:
train_ds = pneumoniaDataset('./data/train/', transform=transformations)
test_ds = pneumoniaDataset('./data/test/', transform=transformations)
val_ds = pneumoniaDataset('./data/val/', transform=transformations)

In [7]:
train_dl = DataLoader(train_ds, batch_size=256, num_workers=8)
test_dl = DataLoader(test_ds, batch_size=256, num_workers=8)
val_dl = DataLoader(val_ds, batch_size=256, num_workers=8)

In [8]:
model = models.resnet50()
layers = []
layers.append(nn.Linear(2048, 1024))
layers.append(nn.ReLU())
layers.append(nn.Linear(1024, 512))
layers.append(nn.ReLU())
layers.append(nn.Linear(512, 1))
layers.append(nn.Sigmoid())
net = nn.Sequential(*layers)
model.fc = net
#torch.nn.Linear(in_features=2048, out_features=1)
model = model.to(device)

In [9]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.05, nesterov=True, weight_decay=0.0005)
criterion = nn.MSELoss(reduce='mean')
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min',patience=3,threshold=0.0001,factor=0.1, verbose=True)

/opt/anaconda3/envs/pytorchenv/lib/python3.9/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [10]:
def train_phase(dl, tr='train'):
    if tr == 'train':
        model.train()
        dl = train_dl
    if tr == 'val':
        model.eval()
        dl = val_dl            
    losses = []
    for i, batch in enumerate(dl):
        inputs, labels = batch['image'], batch['labels'].float().view(-1,1)
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()             # 1. Zero the parameter gradients
        outputs = model(inputs)           # 2. Run the model

        loss = criterion(outputs, labels) # 3. Calculate loss
        losses.append(loss.item())
        #loss = torch.sqrt(loss)           #    -> RMSE loss
        loss.backward()                   # 4. Backward propagate the loss
        optimizer.step()                  # 5. Optimize the network

        del batch
        del inputs
        del labels
        torch.cuda.empty_cache()
    epoch_loss = sum(losses) / len(losses)
    return epoch_loss

In [ ]:
train_losses = []
val_losses = []
epochs = 10
for epoch in range(epochs):
    train_loss = train_phase(train_dl)
    val_loss = train_phase(val_dl, tr='val')
    scheduler.step(val_loss)    
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    print('Epoch {}: Train Loss : {}, Val Loss : {}'.format(epoch, train_loss, val_loss))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(train_losses)
plt.plot(val_losses)
plt.legend(['Train loss', 'Val loss'])
plt.show()

In [ ]:
preds = torch.empty(0, 1)
acts = torch.empty(0,1)
for i, batch in enumerate(test_dl):
    inputs, actuals = batch['image'], batch['labels'].float().view(-1,1)
    inputs, actuals = inputs.to(device), actuals.to(device)
    preds = torch.cat((preds, model(inputs)), axis=0)
    acts = torch.cat((acts, actuals), axis=0)

In [ ]:
preds.shape

In [ ]:
testDf = pd.DataFrame(data=acts.detach().numpy(), columns=['Actual'])
testDf['Preds'] = preds.detach().numpy()
testDf['Preds'] = testDf['Preds'].apply(lambda x: 1 if x > 0.35 else 0)
print(round(testDf[testDf['Actual'] == testDf['Preds']].shape[0]/testDf.shape[0],4))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(testDf['Actual'], testDf['Preds'])

In [4]:
import cv2
image = cv2.imread('./data/train/normal/IM-0656-0001-0001.jpeg')
image.shape

(1966, 2194, 3)

In [6]:
image = cv2.imread('./data/train/normal/IM-0561-0001.jpeg')
image.shape

(1963, 2172, 3)

In [5]:
listdir('./data/train/normal')

['NORMAL2-IM-0927-0001.jpeg',
 'IM-0427-0001.jpeg',
 'IM-0656-0001-0001.jpeg',
 'IM-0561-0001.jpeg',
 'IM-0757-0001.jpeg',
 'NORMAL2-IM-0736-0001.jpeg',
 'NORMAL2-IM-0500-0001.jpeg',
 'NORMAL2-IM-0393-0001.jpeg',
 'NORMAL2-IM-0994-0001.jpeg',
 'IM-0207-0001.jpeg',
 'IM-0494-0001.jpeg',
 'IM-0177-0001.jpeg',
 'IM-0388-0001.jpeg',
 'IM-0341-0001.jpeg',
 'IM-0355-0001.jpeg',
 'IM-0449-0001.jpeg',
 'IM-0480-0001.jpeg',
 'NORMAL2-IM-1038-0001.jpeg',
 'IM-0739-0001.jpeg',
 'IM-0213-0001.jpeg',
 'NORMAL2-IM-0452-0001.jpeg',
 'NORMAL2-IM-0980-0001.jpeg',
 'NORMAL2-IM-0949-0001.jpeg',
 'NORMAL2-IM-0664-0001.jpeg',
 'IM-0269-0001.jpeg',
 'IM-0575-0001.jpeg',
 'IM-0605-0001.jpeg',
 'IM-0433-0001.jpeg',
 'IM-0119-0001.jpeg',
 'NORMAL2-IM-0875-0001.jpeg',
 'NORMAL2-IM-0428-0001.jpeg',
 'NORMAL2-IM-0933-0001.jpeg',
 'NORMAL2-IM-0684-0001-0001.jpeg',
 'NORMAL2-IM-1019-0001.jpeg',
 'IM-0453-0001-0002.jpeg',
 'IM-0374-0001.jpeg',
 'IM-0697-0001.jpeg',
 'IM-0718-0001.jpeg',
 'NORMAL2-IM-0645-0001.jpeg',

In [55]:
create_df = True
if create_df:
    folders = ['train', 'test', 'val']
    classes = ['normal', 'opacity']
    mypath = './data/{}/{}'

    train_normal = [f for f in listdir(mypath.format('train', 'normal')) if isfile(join(mypath.format('train', 'normal'), f))]
    train_normal_label = ['Normal']* len(train_normal)
    train_opacity = [f for f in listdir(mypath.format('train', 'opacity')) if isfile(join(mypath.format('train', 'opacity'), f))]
    train_opacity_label = ['Opacity'] * len(train_opacity)
    
    test_normal = [f for f in listdir(mypath.format('test', 'normal')) if isfile(join(mypath.format('test', 'normal'), f))]
    test_normal_label = ['Normal']* len(test_normal)
    test_opacity = [f for f in listdir(mypath.format('test', 'opacity')) if isfile(join(mypath.format('test', 'opacity'), f))]
    test_opacity_label = ['Opacity'] * len(test_opacity)
    
    
    val_normal = [f for f in listdir(mypath.format('val', 'normal')) if isfile(join(mypath.format('val', 'normal'), f))]
    val_normal_label = ['Normal']* len(val_normal)
    val_opacity = [f for f in listdir(mypath.format('val', 'opacity')) if isfile(join(mypath.format('val', 'opacity'), f))]
    val_opacity_label = ['Opacity'] * len(val_opacity)
    
    train_df = pd.DataFrame(data=train_normal+train_opacity, columns=['id'])
    train_df['label'] = train_normal_label + train_opacity_label
    train_df = pd.get_dummies(train_df, prefix=[''], prefix_sep=[''],columns=['label'])
    
    test_df = pd.DataFrame(data=test_normal+test_opacity, columns=['id'])
    test_df['label'] = test_normal_label + test_opacity_label
    test_df = pd.get_dummies(test_df, prefix=[''], prefix_sep=[''],columns=['label'])
    
    val_df = pd.DataFrame(data=val_normal+val_opacity, columns=['id'])
    val_df['label'] = val_normal_label + val_opacity_label
    val_df = pd.get_dummies(val_df, prefix=[''], prefix_sep=[''],columns=['label'])
    
    train_df.to_csv('./classes/train_df.csv', index=False)
    test_df.to_csv('./classes/test_df.csv', index=False)
    val_df.to_csv('./classes/val_df.csv', index=False)
    